#### TorchScript 有自己的编译器，而不是PYTHON编译器，打破Global Interpreter Lock限制；可以保存模型并且部署到其他设备上；更加高效；可以使用更多其他后端的模型操作；可以直接在C++内使用。

In [2]:
import torch

In [ ]:
#转换方法
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h
#实例化一个模型
my_cell = MyCell()
#设定输入
x, h = torch.rand(3, 4), torch.rand(3, 4)
#转化
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

print(traced_cell.graph)
#更好看，不过这个是具体的执行过程，如果网络中有分支的话，就会被擦掉。
#script compiler可以将分支也表现出来
print(traced_cell.code)

In [ ]:
scripted_gate = torch.jit.script(MyCell())
print(scripted_gate.code)

In [ ]:
# script和trace是可以复合使用的，处理分支的存在
traced.save('wrapped_rnn.pt')
loaded = torch.jit.load('wrapped_rnn.pt')

### STEP1:       先写OP的C++执行文件，称为op.cpp   这个是新版本的
    当前TorchScript 编译器只能使用固定格式数据 
    torch::Tensor, torch::Scalar,  double,   int64_t ,std::vector
### STEP2: 编译 ninja
    JIT

In [ ]:
from torch.utils.cpp_extension import load

module = torch.utils.cpp_extension.load(name='addfunc',
                         sources=['test.cpp'],
                        is_python_module=False,
                        verbose=True
                     )

print(torch.ops.myops.addfunc)

### load_line 


In [ ]:
#!pip install ninja
import torch
import torch.utils.cpp_extension
op_source = """
#include <torch/script.h>
#include <torch/custom_class.h>
#include <iostream>
#include <string>
#include <vector>

// ===================自定义函数=====================
torch::Tensor warp_perspective(const torch::Tensor& image)
{
    return image;
}

// my_ops::warp_perspective 就会表示成torch.ops.myops.warp_perspective       
static auto registry=torch::RegisterOperators("my_ops::warp_perspective",&warp_perspective);
"""
torch.utils.cpp_extension.load_inline(
    name="warp_perspective",
    cpp_sources=op_source,
    is_python_module=False,
    verbose=True,
)
#使用就可以直接导入 import warp_perspective
print(torch.ops.my_ops.warp_perspective)

### 第二种方法 setuptools  
#### setup.py

In [ ]:
from setuptools import setup
from torch.utils.cpp_extension import BuildExtension, CppExtension

setup(
    name="baomingzi",
    ext_modules=[
        CppExtension(
            "mmcv.addfunc",#这是模块路径，生成addfunc.so动态库
            ["test.cpp"],
            libraries=["opencv_core", "opencv_imgproc"],
        )
    ],
    cmdclass={"build_ext": BuildExtension.with_options(no_python_abi_suffix=True)},
)

In [ ]:
#使用方法rt torch
#至关重要！！！！！！！！在build/lib/mmcv/add路径下有add.so，导入即可使用,看清楚模块路径和函数名字的区别
torch.ops.load_library("add.so")
print(torch.ops.my_ops.warp_perspective)


In [ ]:
torch.ops.load_library("libwarp_perspective.so")

@torch.jit.script
def compute(x, y):
    if bool(x[0] == 42):
        z = 5
    else:
        z = 10
    x = torch.ops.my_ops.warp_perspective(x, torch.eye(3))
    return x.matmul(y) + z

# LOADING A TORCHSCRIPT MODEL IN C++  
#### Step 1: Converting Your PyTorch Model to TorchScript

In [ ]:
import torch
from torch import nn
import torchvision
model = torchvision.models.resnet18()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)


#当模型中有if结构的时候，要用stript来处理
class MyModule(nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
            output = self.weight.mv(input)
        else:
            output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

#### Step 2: Serializing Your Script Module to a File

In [ ]:
traced_script_module.save("traced_resnet_model.pt")

#### Step 3: Loading Your Script Module in C++

In [ ]:
mkdir build
cd build
cmake -DCMAKE_PREFIX_PATH=/path/to/libtorch ..#下载这个
cmake --build . --config Release